In [2]:
import numpy as np
import scipy as sp

In [197]:
def rodrigez(fi,p):
    e=np.eye(3)
    px=np.array([[0,-p[2],p[1]],
                 [p[2],0,-p[0]],
                 [-p[1],p[0],0]])
    ppT=np.array([[p[0]**2,p[0]*p[1],p[0]*p[2]]
                 ,[p[1]*p[0],p[1]**2,p[1]*p[2]]
                 ,[p[2]*p[0],p[2]*p[1],p[2]**2]])
    return (1-np.cos(fi))*ppT+np.cos(fi)*e+np.sin(fi)*px

In [214]:
def A2AxisAngle(A):
# 1. Odredi jedinicni sopstveni vektor p za lambda=1
# 2. Odredi proizvoljan jedinicni vektor u normalan na p
# 3. Odredi u_prim koji je jednak A*u (u_prim je jedinicni)
# 4. Odredi fi koji je jednak arccos(u,u_prim)
# 5. Ako je mesoviti proizvod u,u_prim,p < 0, onda p=-p
# 1.
    sopstvene_vrednosti,sopstveni_vektori=sp.linalg.eig(A,left=True,right=False)
    p=sopstveni_vektori[:,2]
    p=np.array(p)
    p=np.real(p)
# 2. Gram Šmit ortogonalizacija
    u = np.array([1.0,2.0,3.0])
    u -= u.dot(p) * p / np.linalg.norm(p)**2 
    u /= np.linalg.norm(u)
# 3.
    u_prim=A@u
    u_prim/=np.linalg.norm(u_prim)
# 4.    
    fi=np.arccos(u.dot(u_prim)/(np.linalg.norm(u)*np.linalg.norm(u_prim)))
# 5.
    if np.dot(u,np.cross(u_prim,p)) < 0:
        p=-p
    return p,fi

In [215]:
A=np.array([[1,-8,-4],[4,4,-7],[8,-1,4]],dtype='float32')
A=A/9
print(A)
p,fi=A2AxisAngle(A)
rodrigez(fi,p)

[[ 0.11111111 -0.8888889  -0.44444445]
 [ 0.44444445  0.44444445 -0.7777778 ]
 [ 0.8888889  -0.11111111  0.44444445]]


array([[ 0.11111117, -0.88888896, -0.44444433],
       [ 0.44444442,  0.44444439, -0.77777785],
       [ 0.88888893, -0.11111099,  0.44444447]])

In [216]:
def A2Euler(A):
    if A[2][0] < 1:
        if A[2][0] > -1: #jedinstveno resenje
            psi=np.arctan2(A[1][0],A[0][0])
            teta=np.arcsin(-A[2][0])
            fi=np.arctan2(A[2][1],A[2][2])
            return psi,teta,fi
        else: # Ox3=-Oz
            psi=np.arctan2(-A[0][1],A[1][1])
            teta=np.pi/2
            fi=0
            return psi,teta,fi
    else: #Ox3=Oz
        psi=np.arctan2(-A[0][1],A[1][1])
        teta=-np.pi/2
        fi=0
        return psi,teta,fi

In [217]:
A2Euler(A)

(1.3258177, -1.0949141, -0.24497867)

--------------------Odavde pocinje cetvrti domaci--------------------

In [132]:
import numpy as np
import scipy as sp
import math

class Quaternion:
    def __init__(self,params,w):
        self.i=params[0]
        self.j=params[1]
        self.k=params[2]
        self.params=params
        self.w=w
        
    def __truediv__(self,o:Quaternion)->Quaternion:
        if isinstance(o,Quaternion):
            donji=o.i**2+o.j**2+o.k**2+o.w**2
            return Quaternion([(o.w*self.i-o.i*self.w-o.j*self.k+o.k*self.j)/donji,
                              (o.w*self.j-o.i*self.k-o.j*self.w+o.k*self.i)/donji,
                              (o.w*self.k-o.i*self.j-o.j*self.i+o.k*self.w)/donji],
                              (o.w*self.w-o.i*self.i-o.j*self.j+o.k*self.k)/donji)
        else:
            for x in self.params:
                x/=o
            return Quaternion(self.params,self.w/o)
            
    def __add__(self,o:Quaternion)->Quaternion:
        return Quaternion([self.i+o.i,self.j+o.j,self.k+o.k],self.w+o.w)

    def __sub__(self,o:Quaternion)->Quaternion:
        return Quaternion([self.i-o.i,self.j-o.j,self.k-o.k],self.w-o.w)       
    
    def __mul__(self,o:Quaternion)->Quaternion:
        # if self.w == 0 and o.w == 0 :
        #     novi_params=np.cross(self.params,o.params)
        #     novo_w=np.dot(self.params,o.params)
        #     return Quaternion([novi_params[0],
        #                       novi_params[1],
        #                       novi_params[2]],
        #                       novo_w)
        # else:
        #     novi_params=np.cross(self.params,o.params)+self.w*o.params+o.w*self.params
        #     novo_w=self.w*o.w-np.dot(self.params,o.params)
        #     return Quaternion([novi_params[0],
        #                       novi_params[1],
        #                       novi_params[2]],
        #                       novo_w)
        if isinstance(o,Quaternion): 
            return Quaternion([self.w*o.i+self.i*o.w+self.j*o.k-self.k*o.j,
                               self.w*o.j+self.j*o.w+self.k*o.i-self.i*o.k,
                               self.w*o.k+self.k*o.w+self.i*o.j-self.j*o.i],
                               self.w*o.w-self.i*o.i-self.j*o.j-self.k*o.k)
        else:
            for x in self.params:
                x*=o
            return Quaternion(self.params,self.w*o)
            
    def __idiv__(self,o:Quaternion)->Quaternion:
            return self/o
        
    def konjugat(self)->Quaternion:
        for x in self.params:
            x=-x
        return Quaternion(self.params,self.w)
        
    def norm(self)->float:
        return np.sqrt(self.i**2+self.j**2+self.k**2+self.w**2)
        
    def dot(self,o:Quaternion)->float:
            return self.i*o.i+self.j*o.j+self.k*o.k+self.w*o.w
        
    def inverse(self)->Quaternion:
        return self.konjugat()/(self.norm()**2)
        
    def __str__(self)->str:
        return '['+str(self.i)+','+str(self.j)+','+str(self.k)+','+str(self.w)+']' 
    def __repr__(self)->str:
        return '['+str(self.i)+','+str(self.j)+','+str(self.k)+','+str(self.w)+']' 

    def __neg__(self)->Quaternion:
        fi,p=Q2AngleAxis(self)
        return AngleAxis2Q(-fi,-p)

    def __eq__(self,o:Quaternion)->bool:
        return (self.i==o.i and self.j==o.j and self.k==o.k and self.w==o.w)

#dodato zbog predstavljanja SLerpa pomocu stepena (videcemo da li radi)
    def vector_part(self)->Quaternion:
        return (self-self.konjugat())/2
    def sign(self)->Quaternion:
        return self/self.norm()
    def arg(self)->float:
        return np.arccos(w/self.norm())
    def exp(self):
        return math.exp(self.w)*(np.cos(self.vector_part().norm())+self.sign(self.vector_part())*np.sin(self.norm(self.vector_part())))
    def ln(self):
        return math.log(self.norm())+self.sign()*np.sin(self.vector_part().norm())
    def __pow__(self,o:int):
        return (self.ln()*n).exp()

In [123]:
 def AngleAxis2Q(fi,p):
        w=np.cos(fi/2)
        p/=np.linalg.norm(p)
        params=np.sin(fi/2)*p
        return Quaternion([params[0],params[1],params[2]],w)

In [111]:
def Q2AngleAxis(q:Quaternion):
    q/=q.norm()
    if q.w<0:
        q=-q
    fi=2*np.arccos(q.w)
    if abs(q.w) == 1:
        p=np.array[1,0,0]
    else:
        p=q.params
        p/=np.linalg.norm(p)
    return fi,p

In [129]:
def SLerp(q1:Quaternion,q2:Quaternion,tm:int):
    cos0=q1.dot(q2)
    print (cos0)
    if cos0 < 0:
        q1=-q1
        cos0=-cos0
    if cos0 > 0.95:
        q1=-q1 #ili lerp interpolaciju
    fi=np.arccos(cos0)
    return [q1*(np.sin((1-i/tm)*fi)/np.sin(fi))
            +q2*(np.sin((i/tm)*fi)/np.sin(fi)) for i in range(tm)]

In [97]:
q1=Quaternion([3,0,-5],1)
q2=Quaternion([1,1,0],7)
(q1*q2).inverse()

[27,-4,-32,0.002240896358543417]

In [118]:
q3=-q1

[-0.51449576 -0.          0.85749293]
-0.9856107606091623
[ 0.50709255  0.         -0.84515425]


In [120]:
q3=q1/q1.norm()
q4=q2/q2.norm()

In [128]:
SLerp(q3,q4,10)

3.1656833739159027


C:\Users\Tanja\AppData\Local\Temp\ipykernel_13520\2674741016.py:9: RuntimeWarning: invalid value encountered in arccos
  fi=np.arccos(cos0)


[[1.5142797168895836,1.0,-0.8571328614826393,nan],
 [1.5142797168895836,1.0,-0.8571328614826393,nan],
 [1.5142797168895836,1.0,-0.8571328614826393,nan],
 [1.5142797168895836,1.0,-0.8571328614826393,nan],
 [1.5142797168895836,1.0,-0.8571328614826393,nan],
 [1.5142797168895836,1.0,-0.8571328614826393,nan],
 [1.5142797168895836,1.0,-0.8571328614826393,nan],
 [1.5142797168895836,1.0,-0.8571328614826393,nan],
 [1.5142797168895836,1.0,-0.8571328614826393,nan],
 [1.5142797168895836,1.0,-0.8571328614826393,nan]]